# REGLAS DE ASOCIACIÓN

##### Autor: 
* Javier Tomás Fernández Martín

## 0. Preliminares

Antes de empezar con el código, importamos todas las librerias que vamos a necesitar.

In [1]:
import pandas as pd
import numpy as np
from apyori import apriori

from sklearn.model_selection import train_test_split

Además de definir una semilla para asegurar que sea reproducible

In [2]:
random_state = 42


Y definimos un par de funciones que usaremos para evaluar módelos con distintos parámetros y compararlos entre sí

## 1.Carga de datos

In [3]:
df = pd.read_csv('bbdd2.csv', index_col=0)


Mostramos una pequeña muestra para asegurar que se ha cargado bien y de paso echarle un vistazo a nuestra BBDD

In [4]:
df.sample(5, random_state=random_state)

,P1_TFT8_Ashe_Tier,P1_TFT8_Ashe_Obj1,P1_TFT8_Ashe_Obj2,P1_TFT8_Ashe_Obj3,P1_TFT8_Blitzcrank_Tier,P1_TFT8_Blitzcrank_Obj1,P1_TFT8_Blitzcrank_Obj2,P1_TFT8_Blitzcrank_Obj3,P1_TFT8_Lucian_Tier,P1_TFT8_Lucian_Obj1,...,P2_TFT8_Urgot_Obj2,P2_TFT8_Urgot_Obj3,P2_TFT8b_Sivir_Tier,P2_TFT8b_Sivir_Obj1,P2_TFT8b_Sivir_Obj2,P2_TFT8b_Sivir_Obj3,P2_Augment1,P2_Augment2,P2_Augment3,P1_Win
1374,0,0,0,0,0,0,0,0,0,0,...,TFT_Item_FrozenHeart,TFT_Item_SeraphsEmbrace,0,0,0,0,TFT6_Augment_ClearMind,TFT6_Augment_JeweledLotus,TFT6_Augment_ComponentGrabBag,0
808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_TradeSector,TFT6_Augment_TomeOfTraits1,TFT8_Augment_MissFortuneSupport,0
4842,0,0,0,0,0,0,0,0,2,0,...,TFT_Item_Deathblade,0,0,0,0,0,TFT6_Augment_SecondWind1,TFT6_Augment_TinyTitans,TFT8_Augment_MissFortuneCarry,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_ForceOfNature,TFT6_Augment_ComponentGrabBag,TFT8_Augment_JannaCarry,1
755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,TFT6_Augment_CelestialBlessing2,TFT7_Augment_AxiomArc2,TFT6_Augment_PortableForge,1


In [5]:

cat_cols = [col for col in df.columns if 'Obj' in col or 'Augment' in col]
df[cat_cols] = df[cat_cols].astype(str)

num_cols = [col for col in df.columns if col not in cat_cols + ['P1_Win']]
cat_cols_idx = [df.columns.get_loc(col) for col in cat_cols]


In [6]:
df_train,df_test = train_test_split(df, test_size=0.2, random_state=random_state)

## 2. Generación de modelos

In [7]:
# Transacciones
transactions = []
for i in range(len(df_train)):
    transaction = []
    for j in range(len(df_train.columns)):
        column_name = df_train.columns[j]
        if "_Tier" in column_name:
            champion_name = column_name.split("_Tier")[0]
            champion_tier = df_train.iloc[i][j]
            if champion_tier > 0:
                transaction.append(f"{champion_name}_Tier {champion_tier}")
        elif "_Obj" in column_name:
            item_name = column_name.split("_Obj")[0]
            item_value = df_train.iloc[i][j]
            if item_value != "0":
                transaction.append(f"{item_name} obj{column_name[-1]} {item_value}")
        elif "Augment" in column_name:
            augment_name = column_name.replace("_", "")
            augment_value = df_train.iloc[i][j]
            if augment_value != "0":
                transaction.append(f"{augment_name} {augment_value}")
    
    
    objective_value = str(df_train.iloc[i][-1])
    if objective_value == "0":
            transaction.append("P1_Lose")
    else:
            transaction.append("P1_Win")
            
    transactions.append(transaction)


In [8]:
# Generar las reglas de asociación
results = list(apriori(transactions, min_support=0.01, min_confidence=0.7, min_lift=3))



In [9]:
# Imprimir las reglas de asociación
for r in results:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1Augment1 TFT8_5_Augment_SivirCarry'] -> ['P1_TFT8_EzrealFuture_Tier 2']: support=0.010256410256410256, confidence=0.7164179104477613, lift=7.500751277171192
['P1Augment1 TFT8_5_Augment_SivirCarry'] -> ['P1_TFT8_Shen_Tier 2']: support=0.01047008547008547, confidence=0.7313432835820896, lift=4.168923955132984
['P1Augment1 TFT8_5_Augment_YasuoCarry'] -> ['P1_TFT8_Yasuo_Tier 3']: support=0.026709401709401708, confidence=0.9842519685039369, lift=17.716535433070867
['P1Augment1 TFT8_Augment_AsheCarry'] -> ['P1_TFT8_Ashe_Tier 3']: support=0.011111111111111112, confidence=0.9629629629629629, lift=46.460481099656356
['P1Augment1 TFT8_Augment_AsheCarry'] -> ['P1_TFT8_Warwick_Tier 2']: support=0.010256410256410256, confidence=0.8888888888888888, lift=6.634768740031898
['P1Augment1 TFT8_Augment_GenAEEmblem'] -> ['P1_TFT8_GnarBig_Tier 3']: support=0.01581196581196581, confidence=0.9135802469135802, lift=13.573192239858907
['P1Augment1 TFT8_Augment_InterPolarisTrait'] -> ['P1_TFT8_Mordekaiser_Ti

In [10]:
final_rules=[]

for rule in results:
    if 'P1_Win' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)
    elif 'P1_Lose' in rule.ordered_statistics[0].items_add:
        final_rules.append(rule)

In [11]:
# Imprimir las reglas de asociación
for r in final_rules:
    print("{} -> {}: support={}, confidence={}, lift={}".format(
        list(r.ordered_statistics[0].items_base), list(r.ordered_statistics[0].items_add),
        r.support, r.ordered_statistics[0].confidence, r.ordered_statistics[0].lift))

['P1Augment1 TFT8_5_Augment_YasuoCarry', 'P1_TFT8_LeeSin_Tier 3'] -> ['P1_TFT8_Yasuo_Tier 3', 'P1_Win']: support=0.010256410256410256, confidence=0.7058823529411764, lift=21.5916955017301
['P1_TFT8_Malphite_Tier 3', 'P1_TFT8_Fiora_Tier 3'] -> ['P1_TFT8_Yasuo_Tier 3', 'P1_Win']: support=0.01047008547008547, confidence=0.7, lift=21.41176470588235
['P1_TFT8_Kaisa obj2 TFT_Item_StatikkShiv', 'P2_TFT8_Alistar_Tier 2', 'P1_TFT8_Neeko_Tier 2'] -> ['P1_TFT8_MissFortune_Tier 2', 'P1_Win']: support=0.01047008547008547, confidence=0.7538461538461538, lift=4.660501981505944
['P1_TFT8_Nilah_Tier 2', 'P1_TFT8_Garen_Tier 2', 'P1_TFT8_Kaisa_Tier 3', 'P1_TFT8_Rell_Tier 3'] -> ['P1_TFT8_Ekko_Tier 2', 'P1_Win']: support=0.01047008547008547, confidence=0.7101449275362318, lift=3.516908212560386
['P1_TFT8_Nilah_Tier 2', 'P1_TFT8_Ekko_Tier 2', 'P1_TFT8_Rell_Tier 3', 'P1_TFT8_Kaisa_Tier 3'] -> ['P1_TFT8_MissFortune_Tier 2', 'P1_Win']: support=0.01517094017094017, confidence=0.7171717171717171, lift=4.4337696

## 3. EVALUACIÓN DE MODELOS

In [138]:
def score_model(transactions, model_rules):
    """
    Calcula el score del modelo basado en reglas obtenido a partir de un dataframe y un modelo de reglas.

    :param df: Pandas DataFrame con el conjunto de datos.
    :param model_rules: Lista de reglas obtenidas a partir de Apriori.
    :return: Float con la precisión del modelo.
    """


    # Calcular las predicciones del modelo para cada transacción
    predictions = []
    for transaction in transactions:
        transaction_set = set([f"{column}={value}" for column, value in enumerate(transaction)])
        for rule in model_rules:
            if rule[0].issubset(transaction_set):
                predictions.append(1 if "P1_Win=1" in rule else 0)
                break
        else:
            # Si no se encuentra ninguna regla que se aplique a la transacción, predecir 0
            predictions.append(0)

    # Calcular la precisión del modelo
    actual_values = df["P1_Win"].values.tolist()
    correct_predictions = sum([1 if p == a else 0 for p, a in zip(predictions, actual_values)])
    accuracy = correct_predictions / len(predictions)

    return accuracy

In [139]:
score_model(transactions, final_rules)

0.5017595307917888